In [1]:
import numpy as np
import pandas as pd
import linecache as gl
import glob
import os
import platform
op_sys = platform.system()
if op_sys == 'Windows':
    dtdir = 'C:\\Users\\kurt_\\Google Drive\\dell_github\\T-Matrix-Titan\\special\\data\\output\\'
    _sl = '\\'
elif op_sys == 'Linux':
    dtdir = '/home/cihat/Dropbox/code/special/data/'
    _sl = '/'
os.chdir(dtdir)


In [2]:
tmf_list = pd.read_csv(dtdir+'output_list.csv',dtype=str)
tmf_list.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1031 entries, 0 to 1030
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   N       1031 non-null   object
 1   xm      1031 non-null   object
 2   nr      1031 non-null   object
 3   ni      1031 non-null   object
dtypes: object(4)
memory usage: 32.3+ KB


In [3]:
_log = []
tm_all_mx = []
tm_all_eff = []
count = 0
def convert_str(i, _var):
    var = tmf_list.iloc[i][_var]
    var = "".join([x for x in str(var) if x != "."])
    return var
for i in tmf_list.index:
    N = convert_str(i, "N")
    xm = "x"+convert_str(i, "xm")
    nr = convert_str(i, "nr")
    ni = convert_str(i, "ni")

    # We need to skip the loop if there is no data for this ni.
    try:
        tmatrix_files = glob.glob(
            dtdir+_sl.join([N,xm,nr,ni,''])+"mt*.dat")
        tmatrix_files[0]
    except IndexError:
        _log.append(['There is no data for this set:  {}_{}_{}_'
                        .format(N, xm, nr) + ni])
        continue
    # Skipping runs if there are not atleast 10 outputs available.

    if len(tmatrix_files) < 5 and len(tmatrix_files) > 0:

        print('There are {} outputs for the run {}_{}_{}_'
                .format(len(tmatrix_files), N, xm, nr) + ni)
        print('For the sake of accuracy this run will not be included ')
        continue
    elif len(tmatrix_files) == 0:
        print('There is NO outputs for the run {}_{}_{}_'
                .format(N, xm, nr) + ni)
        continue
    print('There are {} output files for the run {}_{}_{}_'
                    .format(len(tmatrix_files), N, xm, nr) + ni)
    i = 0
    df_mx_concat = []
    df_eff_concat = []
    for tmf in tmatrix_files:
        ## Skip Incomplete output files ##
        file_size = os.path.getsize(tmf)

        if file_size < 2000:
            print(file_size)
            continue

        i += 1
        # Let's read the mstm output files and save the data into
        # a pd.DataFrame
        # Outside this loop, I will calculate the ensemble average.

        # Let's first collect the efficiencies and asymmetry parm.

        q = gl.getline(tmf, 51)
        q = q.split()
        qext = [float(q[0])]
        qabs = [float(q[1])]
        qsca = [float(q[2])]
        g = [float(q[3])]
        # A few runs seems like have wrong qe and/or qa values.
        # I did not investigate the scattering matrix but I will
        # completely skip all these runs.

        if (qsca[0] < 1e3 and qsca[0] > -1e3) and (qabs[0] < 1e3 and qabs[0] > -1e3)\
                and (g[0] >= 0 and g[0] <= 1):
            eff = {'qa': qabs, 'qs': qsca, 'qe': qext, 'g': g}
            dt_eff = pd.DataFrame(eff)
            dt_eff = dt_eff.apply(pd.to_numeric)
            df_eff_concat.append(dt_eff)

            data = []
            for line in range(54, 235):
                a = gl.getline(tmf, line)
                columns = a.split()
                rw = {}
                rw['theta'] = columns[0]
                rw['P11'] = columns[1]
                rw['P12'] = columns[2]
                rw['P13'] = columns[3]
                rw['P14'] = columns[4]
                rw['P22'] = columns[5]
                rw['P23'] = columns[6]
                rw['P24'] = columns[7]
                rw['P33'] = columns[8]
                rw['P34'] = columns[9]
                rw['P44'] = columns[10]
                data.append(rw)

            # Dataframe for each run with a name, e.g., tm1024_x3360_171_009_1
            dt_mx = pd.DataFrame(data)

            # Let's convert the string values to numeric all at once
            dt_mx = dt_mx.apply(pd.to_numeric)

            # Now lets collect all the dataframes to get an average at the end.

            df_mx_concat.append(dt_mx)
        else:
            continue

    # Now let's average the all runs for the same input parameters
    # df_concat stack all the datframes vertically.
    # indexes, 1 to 180, repeats for each output
    # To calculate the mean we just need to group the
    # df_concat by the index.
    # then we can simply take the mean.

    # Let's convert the df_concat to a pandas dataframe
    df_mx_concat = pd.concat(df_mx_concat)
    df_eff_concat = pd.concat(df_eff_concat)

    # Let's take the average of all runs for each set

    # Averaging Scattering matrices
    group_by_index_mx = df_mx_concat.groupby(df_mx_concat.index)
    tm_run_mx = 'tm{}_{}_{}_{}_mx'.format(N, xm, nr, ni)
    vars()[tm_run_mx] = group_by_index_mx.mean()
    # For eg. 1024_x6398_13_009_mx
    tm_all_mx.append(vars()[tm_run_mx])

    # Averaging Efficiencies and asym. par.
    group_by_index_eff = df_eff_concat.groupby(df_eff_concat.index)
    tm_run_eff = 'tm{}_{}_{}_{}_eff'.format(N, xm, nr, ni)
    vars()[tm_run_eff] = group_by_index_eff.mean()
    tm_all_eff.append(vars()[tm_run_eff])
    count+=1


There are 25 output files for the run 1024_x03360_130_00010
There are 25 output files for the run 1024_x03360_130_00040
There are 25 output files for the run 1024_x03360_130_00230
There are 25 output files for the run 1024_x03360_130_00450
There are 25 output files for the run 1024_x03360_130_00900
There are 25 output files for the run 1024_x03360_166_00010
There are 25 output files for the run 1024_x03360_166_00020
There are 25 output files for the run 1024_x03360_166_00020
There are 25 output files for the run 1024_x03360_171_00230
There are 25 output files for the run 1024_x03360_171_00900
There are 25 output files for the run 1024_x03360_200_00040
1458
There are 25 output files for the run 1024_x03360_200_00230
There are 25 output files for the run 1024_x03360_200_00450
There are 25 output files for the run 1024_x03360_200_00900
There are 25 output files for the run 1024_x04300_130_00230
There are 25 output files for the run 1024_x04300_130_00900
There are 25 output files for the r

ValueError: No objects to concatenate

In [7]:
df_mx_concat 

[]

In [58]:
tmatrix_files

['C:\\Users\\kurt_\\Google Drive\\dell_github\\T-Matrix-Titan\\special\\data\\output\\1024\\x03360\\130\\00040\\mt_10_1024_03360_130_00040.dat',
 'C:\\Users\\kurt_\\Google Drive\\dell_github\\T-Matrix-Titan\\special\\data\\output\\1024\\x03360\\130\\00040\\mt_11_1024_03360_130_00040.dat',
 'C:\\Users\\kurt_\\Google Drive\\dell_github\\T-Matrix-Titan\\special\\data\\output\\1024\\x03360\\130\\00040\\mt_12_1024_03360_130_00040.dat',
 'C:\\Users\\kurt_\\Google Drive\\dell_github\\T-Matrix-Titan\\special\\data\\output\\1024\\x03360\\130\\00040\\mt_13_1024_03360_130_00040.dat',
 'C:\\Users\\kurt_\\Google Drive\\dell_github\\T-Matrix-Titan\\special\\data\\output\\1024\\x03360\\130\\00040\\mt_14_1024_03360_130_00040.dat',
 'C:\\Users\\kurt_\\Google Drive\\dell_github\\T-Matrix-Titan\\special\\data\\output\\1024\\x03360\\130\\00040\\mt_15_1024_03360_130_00040.dat',
 'C:\\Users\\kurt_\\Google Drive\\dell_github\\T-Matrix-Titan\\special\\data\\output\\1024\\x03360\\130\\00040\\mt_16_1024_03360_1

In [64]:
i

25

In [48]:
qs = glob.glob(
            dtdir+_sl.join([N,xm,nr,ni,''])+"*.dat")
qs

['C:\\Users\\kurt_\\Google Drive\\dell_github\\T-Matrix-Titan\\special\\data\\output\\1024\\x03360\\130\\00040\\mt_10_1024_03360_130_00040.dat',
 'C:\\Users\\kurt_\\Google Drive\\dell_github\\T-Matrix-Titan\\special\\data\\output\\1024\\x03360\\130\\00040\\mt_11_1024_03360_130_00040.dat',
 'C:\\Users\\kurt_\\Google Drive\\dell_github\\T-Matrix-Titan\\special\\data\\output\\1024\\x03360\\130\\00040\\mt_12_1024_03360_130_00040.dat',
 'C:\\Users\\kurt_\\Google Drive\\dell_github\\T-Matrix-Titan\\special\\data\\output\\1024\\x03360\\130\\00040\\mt_13_1024_03360_130_00040.dat',
 'C:\\Users\\kurt_\\Google Drive\\dell_github\\T-Matrix-Titan\\special\\data\\output\\1024\\x03360\\130\\00040\\mt_14_1024_03360_130_00040.dat',
 'C:\\Users\\kurt_\\Google Drive\\dell_github\\T-Matrix-Titan\\special\\data\\output\\1024\\x03360\\130\\00040\\mt_15_1024_03360_130_00040.dat',
 'C:\\Users\\kurt_\\Google Drive\\dell_github\\T-Matrix-Titan\\special\\data\\output\\1024\\x03360\\130\\00040\\mt_16_1024_03360_1

In [35]:
dtdir+_sl.join([N,xm,nr,ni,''])

'C:\\Users\\kurt_\\Google Drive\\dell_github\\T-Matrix-Titan\\special\\data\\output\\1024\\x03360\\130\\00040\\'

In [33]:
path='C:\\Users\\kurt_'
glob.glob('C:\\Users\\kurt_\\Google Drive\\dell_github\\T-Matrix-Titan\\special\\data\\output\\1024\\x03360\\130\\00040\\*')

['C:\\Users\\kurt_\\Google Drive\\dell_github\\T-Matrix-Titan\\special\\data\\output\\1024\\x03360\\130\\00040\\mt_10_1024_03360_130_00040.dat',
 'C:\\Users\\kurt_\\Google Drive\\dell_github\\T-Matrix-Titan\\special\\data\\output\\1024\\x03360\\130\\00040\\mt_11_1024_03360_130_00040.dat',
 'C:\\Users\\kurt_\\Google Drive\\dell_github\\T-Matrix-Titan\\special\\data\\output\\1024\\x03360\\130\\00040\\mt_12_1024_03360_130_00040.dat',
 'C:\\Users\\kurt_\\Google Drive\\dell_github\\T-Matrix-Titan\\special\\data\\output\\1024\\x03360\\130\\00040\\mt_13_1024_03360_130_00040.dat',
 'C:\\Users\\kurt_\\Google Drive\\dell_github\\T-Matrix-Titan\\special\\data\\output\\1024\\x03360\\130\\00040\\mt_14_1024_03360_130_00040.dat',
 'C:\\Users\\kurt_\\Google Drive\\dell_github\\T-Matrix-Titan\\special\\data\\output\\1024\\x03360\\130\\00040\\mt_15_1024_03360_130_00040.dat',
 'C:\\Users\\kurt_\\Google Drive\\dell_github\\T-Matrix-Titan\\special\\data\\output\\1024\\x03360\\130\\00040\\mt_16_1024_03360_1

In [30]:
glob.glob('C:\\Users\\kurt_\\Google Drive\\dell_github\\T-Matrix-Titan\\special\\data\\output\\128\\x04000\\150\\04030\\*')

['C:\\Users\\kurt_\\Google Drive\\dell_github\\T-Matrix-Titan\\special\\data\\output\\128\\x04000\\150\\04030\\mt_11_128_04000_150_04030.dat',
 'C:\\Users\\kurt_\\Google Drive\\dell_github\\T-Matrix-Titan\\special\\data\\output\\128\\x04000\\150\\04030\\mt_13_128_04000_150_04030.dat',
 'C:\\Users\\kurt_\\Google Drive\\dell_github\\T-Matrix-Titan\\special\\data\\output\\128\\x04000\\150\\04030\\mt_15_128_04000_150_04030.dat',
 'C:\\Users\\kurt_\\Google Drive\\dell_github\\T-Matrix-Titan\\special\\data\\output\\128\\x04000\\150\\04030\\mt_17_128_04000_150_04030.dat',
 'C:\\Users\\kurt_\\Google Drive\\dell_github\\T-Matrix-Titan\\special\\data\\output\\128\\x04000\\150\\04030\\mt_19_128_04000_150_04030.dat',
 'C:\\Users\\kurt_\\Google Drive\\dell_github\\T-Matrix-Titan\\special\\data\\output\\128\\x04000\\150\\04030\\mt_1_128_04000_150_04030.dat',
 'C:\\Users\\kurt_\\Google Drive\\dell_github\\T-Matrix-Titan\\special\\data\\output\\128\\x04000\\150\\04030\\mt_3_128_04000_150_04030.dat',
 